## 補足 東京JR中央線マンション価格 - Tabpy経由でリアルタイム予測
![tabpy](../images/tabpy連携image.png)

#### ライブラリのインポートとデータの準備
必要なライブラリを呼び出します。

In [1]:
import pandas as pd 
import numpy as np

Tableau Prep Builder で前準備をした入力変数をデータフレームに読み込みます

In [2]:
df = pd.read_csv('housing_tokyo_chuo_line_prep.csv')

データフレームの中身を確認します。

In [3]:
df.head()

,uid,years,minutes,sqrm,distance,renovate,express,rooms,price
0,13101-1,34,3,35,2.983801,0,0,1LDK,24000000
1,13101-2,37,4,20,2.983801,0,0,1K-1DK,13000000
2,13101-4,34,3,30,2.983801,0,0,1K-1DK,22000000
3,13101-5,35,1,70,2.983801,0,0,2LDK,69000000
4,13101-6,34,3,25,2.983801,0,0,1K-1DK,21000000


出力変数(目的変数:t はpriceです。　データ型も整数であることを確認しておきます。


In [4]:
t = df['price']

In [5]:
t.dtypes

dtype('int64')

レコード数を確認します。

In [6]:
t.shape

(2999,)

UIDとpriceを切り取ったものが入力変数:xとなります。

In [7]:
x = df.iloc[:,1:-1]

入力変数:x の中身を確認します。


In [8]:
x.head()

,years,minutes,sqrm,distance,renovate,express,rooms
0,34,3,35,2.983801,0,0,1LDK
1,37,4,20,2.983801,0,0,1K-1DK
2,34,3,30,2.983801,0,0,1K-1DK
3,35,1,70,2.983801,0,0,2LDK
4,34,3,25,2.983801,0,0,1K-1DK


rooms のみobjectでその他は数値型となっています。

In [9]:
x.dtypes

years         int64
minutes       int64
sqrm          int64
distance    float64
renovate      int64
express       int64
rooms        object
dtype: object

ダミー変数化します。この場合、object 型である rooms のみがダミー化されます。

In [10]:
x = pd.get_dummies(x)

中身を確認します。11列あることを確認します。

In [11]:
x.head()

,years,minutes,sqrm,distance,renovate,express,rooms_1K-1DK,rooms_1LDK,rooms_2K-2DK,rooms_2LDK,rooms_3KUP
0,34,3,35,2.983801,0,0,0,1,0,0,0
1,37,4,20,2.983801,0,0,1,0,0,0,0
2,34,3,30,2.983801,0,0,1,0,0,0,0
3,35,1,70,2.983801,0,0,0,0,0,1,0
4,34,3,25,2.983801,0,0,1,0,0,0,0


#### 訓練データと検証データに分割
ここでは、「1.3.機械学習の基本」で学習した、 **ホールドアウト法**を用いてデータを訓練データと検証データに分割します。
`sklearn`の中の`model_selection`の中に`train_test_split`がありますので、こちらをimportしましょう。

In [12]:
from sklearn.model_selection import train_test_split

それぞれ以下の変数名で定義します。
- `x`の訓練データ：`x_train`
- `t`の訓練データ：`t_train`
- `x`の検証データ：`x_val`
- `t`の検証データ：`t_val`

各データのshapeも確認しておきましょう。

In [13]:
# データの訓練データとテストデータに分けます。
x_train, x_val, t_train, t_val = train_test_split(x, t, test_size=0.3, random_state=0)
print('訓練データ : ', x_train.shape, t_train.shape)
print('検証データ : ', x_val.shape, t_val.shape)

訓練データ :  (2099, 11) (2099,)
検証データ :  (900, 11) (900,)


#### scikit-learnのimport
データの準備ができたのでscikit-learnをimportしましょう。`sklearn`の中の`linear_model`（線形モデル）の中の`LinearRegression`（線形回帰）をimportします。

In [14]:
from sklearn.linear_model import LinearRegression

#### モデルの宣言
線形回帰モデルを宣言します。

In [15]:
model = LinearRegression()

#### モデルの学習
学習には訓練データを用います。

In [16]:
model.fit(x_train, t_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

#### モデルの評価
過学習が起きていないことを確認するために、モデルの評価は訓練データ、検証データの両方で行います。

In [17]:
# 訓練データでの決定係数を確認します。
model.score(x_train, t_train)

0.7944561801191815

In [18]:
# 検証データでの決定係数を確認します。
model.score(x_val, t_val)

0.7984318328546784

#### 予測値の計算（推論）
今回は精度を求める試みはせず、このくらいにしておきましょう。
次に、生成されたモデルを使って全てのデータの入力変数を用いて価格を推論します。

In [19]:
y = model.predict(x)

分かりやすいように列名を付けます。

In [20]:
y = pd.DataFrame(y,columns=["predict"])

中身を確認します。

In [21]:
y.head()

,predict
0,3.133670e+07
1,1.672240e+07
2,2.676520e+07
3,6.268844e+07
4,2.291930e+07


読み込んだデータの右端に推論結果を追加します。

In [22]:
results = pd.concat([df,y],axis=1)

結果を確認します。

In [23]:
results.head()

,uid,years,minutes,sqrm,distance,renovate,express,rooms,price,predict
0,13101-1,34,3,35,2.983801,0,0,1LDK,24000000,3.133670e+07
1,13101-2,37,4,20,2.983801,0,0,1K-1DK,13000000,1.672240e+07
2,13101-4,34,3,30,2.983801,0,0,1K-1DK,22000000,2.676520e+07
3,13101-5,35,1,70,2.983801,0,0,2LDK,69000000,6.268844e+07
4,13101-6,34,3,25,2.983801,0,0,1K-1DK,21000000,2.291930e+07


ここからユニークID：uid　と　推論値　predict のみ取り出しCSVに書き出します。（後ほどTableau でオリジナルデータと結合して分析します。）

In [24]:
results[['uid','predict']].to_csv('result_uid_predict.csv')

## 3.2.6. Tabpy Serverに推論モデルをデプロイする

tabpy clientのインポートと準備をします。

In [25]:
import tabpy_client
client = tabpy_client.Client('http://localhost:9004/')

Tabpy Serverにデプロイしコールできる方の関数を定義します。

In [26]:
#Tableau から入力を受け取り推論値を返すための関数「housing price」を定義します。
def housingprice(years, minutes, sqrm, distance, renovate, express, rooms_1k_1dk, rooms_1ldk, rooms_2k_2dk, rooms_2ldk, rooms_3kup):

#pandasのインポート
    import pandas as pd

#入力変数をデータフレームとして格納
    years = pd.DataFrame(years)
    minutes = pd.DataFrame(minutes)
    sqrm = pd.DataFrame(sqrm)
    distance = pd.DataFrame(distance)
    renovate = pd.DataFrame(renovate)
    express = pd.DataFrame(express)
    rooms_1k_1dk = pd.DataFrame(rooms_1k_1dk)
    rooms_1ldk = pd.DataFrame(rooms_1ldk)
    rooms_2k_2dk = pd.DataFrame(rooms_2k_2dk)
    rooms_2ldk = pd.DataFrame(rooms_2ldk)
    rooms_3kup = pd.DataFrame(rooms_3kup)

    #入力変数を１つのデータフレームに連結
    data = pd.concat([years, minutes, sqrm, distance, renovate, express, rooms_1k_1dk, rooms_1ldk, rooms_2k_2dk, rooms_2ldk, rooms_3kup],axis=1)
    
    #作成したモデルを利用して予測値を取得
    y = model.predict(data)
    
    #予測結果をリストとして返す
    return y.tolist()

Tabpy Server にデプロイします。Jupyter Notebook,Tabpyともに管理者権限できどうしないと失敗するようです。

In [37]:
client.deploy('housingprice', housingprice, 'Predict Real Estate Price',override=True)

Tableau から入力を受け付けると想定してそれぞれのパラメータをリストに変換します。一つ一つがTableau のメジャーになるイメージです。

In [38]:
import numpy as np

years = x['years'].values.tolist()
minutes = x['minutes'].values.tolist()
sqrm = x['sqrm'].values.tolist()
distance = x['distance'].values.tolist()
renovate = x['renovate'].values.tolist()
express = x['express'].values.tolist()
rooms_1k_1dk = x['rooms_1K-1DK'].values.tolist()
rooms_1ldk = x['rooms_1LDK'].values.tolist()
rooms_2k_2dk = x['rooms_2K-2DK'].values.tolist()
rooms_2ldk = x['rooms_2LDK'].values.tolist()
rooms_3kup = x['rooms_3KUP'].values.tolist()

client.qury によってデプロイされたモデルにテストデータの入ったリストを投入します。

In [39]:
results = client.query('housingprice',years, minutes, sqrm, distance, renovate, express, rooms_1k_1dk, rooms_1ldk, rooms_2k_2dk,rooms_2ldk,rooms_3kup)

In [40]:
#入力変数をデータフレームとして格納
years = pd.DataFrame(years)
minutes = pd.DataFrame(minutes)
sqrm = pd.DataFrame(sqrm)
distance = pd.DataFrame(distance)
renovate = pd.DataFrame(renovate)
express = pd.DataFrame(express)
rooms_1k_1dk = pd.DataFrame(rooms_1k_1dk)
rooms_1ldk = pd.DataFrame(rooms_1ldk)
rooms_2k_2dk = pd.DataFrame(rooms_2k_2dk)
rooms_2ldk = pd.DataFrame(rooms_2ldk)
rooms_3dk_3ldk = pd.DataFrame(rooms_3kup)

#入力変数を１つのデータフレームに連結
data = pd.concat([years, minutes, sqrm, distance, renovate, express, rooms_1k_1dk, rooms_1ldk, rooms_2k_2dk, rooms_2ldk, rooms_3dk_3ldk],axis=1)


In [41]:
data.shape

(2999, 11)

In [42]:
data.head()

,0,0,0,0,0,0,0,0,0,0,0
0,34,3,35,2.983801,0,0,0,1,0,0,0
1,37,4,20,2.983801,0,0,1,0,0,0,0
2,34,3,30,2.983801,0,0,1,0,0,0,0
3,35,1,70,2.983801,0,0,0,0,0,1,0
4,34,3,25,2.983801,0,0,1,0,0,0,0


In [43]:
y_pred = model.predict(data)

結果を確認します。それなりの値が入っている気がします。

In [44]:
results["response"]

[31336700.812907223,
 16722397.227125943,
 26765196.473031756,
 62688442.88107689,
 22919298.277486183,
 70175370.49630588,
 39243016.523220435,
 53716270.468476884,
 27375019.09504606,
 53657575.017075,
 27375019.09504606,
 53716270.468476884,
 57101364.45273095,
 51150748.294440165,
 19073400.081940606,
 72740892.6703426,
 81713065.0829426,
 53716270.468476884,
 51150748.294440165,
 48352518.177133426,
 69127702.41806704,
 22019394.95161715,
 70175370.49630588,
 27440024.093201786,
 24874501.919165064,
 31578803.360816628,
 32304353.12528944,
 32304353.12528944,
 32304353.12528944,
 17880416.125138886,
 14034517.92959331,
 32304353.12528944,
 32304353.12528944,
 32304353.12528944,
 12920095.28450222,
 31578803.360816628,
 12920095.28450222,
 16765993.480047796,
 20611891.675593372,
 32304353.12528944,
 28685982.809221346,
 12920095.28450222,
 48373756.34369601,
 37662174.49201523,
 74806192.18131399,
 46418460.92403726,
 49925475.22298185,
 27856281.63188288,
 27856281.63188288,
 499

client._get_endpoints() でデプロイされた関数（エンドポイント）の情報を確認します。

In [45]:
client.get_endpoints()['housingprice']

{'type': 'model', 'schema': None, 'creation_time': datetime.datetime(2020, 3, 10, 7, 13), 'dependencies': [], 'last_modified_time': datetime.datetime(2020, 4, 1, 6, 50, 9), 'description': 'Predict Real Estate Price', 'name': 'housingprice', 'version': 5}

### ここからはTableau Desktopで確認し、リアルタイムで物件情報を変更したときに予測値が変わることを確認します。
![dashboard](../images/中央線マンション価格予測ダッシュボード.png)